In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive') 

baseDirectory = '/content/drive/My Drive/FYP - CAR DAMAGE DETECTION/Dataset/custom_dataset_editted'

trainDirectory = os.path.join(baseDirectory, 'Training')
validationDirectory = os.path.join(baseDirectory, 'Testing')

IMAGE_SIZE = 128
BATCH_SIZE = 32

trainDatagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=40, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, vertical_flip=True, fill_mode="nearest")

trainGenerator = trainDatagen.flow_from_directory(directory=trainDirectory, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

validationDatagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validationGenerator = validationDatagen.flow_from_directory(directory=validationDirectory, target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

baseModel = tf.keras.applications.VGG19(input_shape=IMG_SHAPE, include_top=False)
baseModel.trainable = True

baseModel = tf.keras.Sequential([
  baseModel,
  #keras.layers.Conv2D(128, (3, 3), activation='sigmoid', input_shape=IMG_SHAPE),
  keras.layers.GlobalMaxPool2D(),
  keras.layers.Dense(4, activation='sigmoid')

])

baseModel.summary()

baseModel.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

epochs = 50
stepsPerEpoch = numpy.ceil(trainGenerator.n / BATCH_SIZE)
validationSteps = numpy.ceil(validationGenerator.n / BATCH_SIZE)

history = baseModel.fit(trainGenerator,
                              steps_per_epoch = stepsPerEpoch,
                              epochs=epochs,
                              validation_data=validationGenerator,
                              validation_steps=validationSteps)

acc = history.history['accuracy']
valAcc = history.history['val_accuracy']

loss = history.history['loss']
valLoss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(valAcc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(valLoss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

# baseModel.save('MobileNet_TransferLearning_DamageCar.h5')